### Exploring The Impact of Optimizers and Activation Functions On OODN 

In [4]:
from os import listdir

import time
import json

import torch

from torchvision.datasets import mnist, FashionMNIST
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torch.optim import SGD, Adam
from torch.nn import Module
from torch import nn
from torch.nn import CrossEntropyLoss
from torchvision.models.resnet import Bottleneck, ResNet

from wilds import get_dataset
from wilds.common.data_loaders import get_train_loader
import torchvision.transforms as transforms

import numpy as np
import pandas as pd

from openood.evaluators import metrics

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


### Supported Activation Functions

For activation functions, we are considering ReLU, Softplus, Swish. *Note that we may conduct experiments for a subset based on the compute resources available*

In [6]:
def get_activation_fn(activation):
    if activation == 'relu':
        return nn.ReLU()
    elif activation == 'softplus':
        return nn.Softplus()
    elif activation == 'swish':
        return nn.Swish()
    return None

### Supported Networks

Currently, we support LeNet and ResNet50.

### LeNet

In [7]:
class LeNet(nn.Module):
    def __init__(self, num_classes, num_channel=3, activation='relu'):
        super(LeNet, self).__init__()
        self.num_classes = num_classes
        self.feature_size = 84
        self.block1 = nn.Sequential(
            nn.Conv2d(in_channels=num_channel,
                      out_channels=6,
                      kernel_size=5,
                      stride=1,
                      padding=2), get_activation_fn(activation), nn.MaxPool2d(kernel_size=2))

        self.block2 = nn.Sequential(
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
             get_activation_fn(activation), nn.MaxPool2d(kernel_size=2))

        self.block3 = nn.Sequential(
            nn.Conv2d(in_channels=16,
                      out_channels=120,
                      kernel_size=5,
                      stride=1), get_activation_fn(activation))

        self.classifier1 = nn.Linear(in_features=120, out_features=84)
        self.relu = get_activation_fn(activation)
        self.fc = nn.Linear(in_features=84, out_features=num_classes)

    def get_fc(self):
        fc = self.fc
        return fc.weight.cpu().detach().numpy(), fc.bias.cpu().detach().numpy()

    def forward(self, x, return_feature=False, return_feature_list=False):
        feature1 = self.block1(x)
        feature2 = self.block2(feature1)
        feature3 = self.block3(feature2)
        feature3 = feature3.view(feature3.shape[0], -1)
        feature = self.relu(self.classifier1(feature3))
        logits_cls = self.fc(feature)
        feature_list = [feature1, feature2, feature3, feature]
        if return_feature:
            return logits_cls, feature
        elif return_feature_list:
            return logits_cls, feature_list
        else:
            return logits_cls

    def forward_threshold(self, x, threshold):
        feature1 = self.block1(x)
        feature2 = self.block2(feature1)
        feature3 = self.block3(feature2)
        feature3 = feature3.view(feature3.shape[0], -1)
        feature = self.relu(self.classifier1(feature3))
        feature = feature.clip(max=threshold)
        logits_cls = self.fc(feature)

        return logits_cls

### ResNet50

In [8]:
class ResNet50(ResNet):
    def __init__(self,
                 block=Bottleneck,
                 layers=[3, 4, 6, 3],
                 num_classes=1000):
        super(ResNet50, self).__init__(block=block,
                                       layers=layers,
                                       num_classes=num_classes)
        self.feature_size = 2048


    def forward(self, x, return_feature=False, return_feature_list=False):
        feature1 = self.relu(self.bn1(self.conv1(x)))
        feature1 = self.maxpool(feature1)
        feature2 = self.layer1(feature1)
        feature3 = self.layer2(feature2)
        feature4 = self.layer3(feature3)
        feature5 = self.layer4(feature4)
        feature5 = self.avgpool(feature5)
        feature = feature5.view(feature5.size(0), -1)
        logits_cls = self.fc(feature)

        feature_list = [feature1, feature2, feature3, feature4, feature5]
        if return_feature:
            return logits_cls, feature
        elif return_feature_list:
            return logits_cls, feature_list
        else:
            return logits_cls

    def forward_threshold(self, x, threshold):
        feature1 = self.relu(self.bn1(self.conv1(x)))
        feature1 = self.maxpool(feature1)
        feature2 = self.layer1(feature1)
        feature3 = self.layer2(feature2)
        feature4 = self.layer3(feature3)
        feature5 = self.layer4(feature4)
        feature5 = self.avgpool(feature5)
        feature = feature5.clip(max=threshold)
        feature = feature.view(feature.size(0), -1)
        logits_cls = self.fc(feature)

        return logits_cls

    def get_fc(self):
        fc = self.fc
        return fc.weight.cpu().detach().numpy(), fc.bias.cpu().detach().numpy()
    
def get_resnet_model(activation_function_type, n_classes):
    resnet_model = ResNet50(num_classes=n_classes)
    resnet_model.to(device)    
    return resnet_model

def set_activation_function(resnet_model, activation_function_type):
    resnet_model.relu = get_activation_fn(activation_function_type)
    resnet_model.layer1[0].relu = get_activation_fn(activation_function_type)
    resnet_model.layer1[1].relu = get_activation_fn(activation_function_type)
    resnet_model.layer1[2].relu = get_activation_fn(activation_function_type)

    resnet_model.layer2[0].relu = get_activation_fn(activation_function_type)
    resnet_model.layer2[1].relu = get_activation_fn(activation_function_type)
    resnet_model.layer2[2].relu = get_activation_fn(activation_function_type)
    resnet_model.layer2[3].relu = get_activation_fn(activation_function_type)

    resnet_model.layer3[0].relu = get_activation_fn(activation_function_type)
    resnet_model.layer3[1].relu = get_activation_fn(activation_function_type)
    resnet_model.layer3[2].relu = get_activation_fn(activation_function_type)
    resnet_model.layer3[3].relu = get_activation_fn(activation_function_type)
    resnet_model.layer3[4].relu = get_activation_fn(activation_function_type)
    resnet_model.layer3[5].relu = get_activation_fn(activation_function_type)


    resnet_model.layer4[0].relu = get_activation_fn(activation_function_type)
    resnet_model.layer4[1].relu = get_activation_fn(activation_function_type)
    resnet_model.layer4[2].relu = get_activation_fn(activation_function_type)
    
    return resnet_model

In [9]:
def get_model(config):
    activation_function_type = config["activation_function_type"]
    network_type = config["network"]
    n_classes = config["n_classes"]
    
    if network_type == "lenet":
        model =  LeNet(num_classes=n_classes, num_channel=1, activation=activation_function_type)
    elif network_type == "resnet50":
        model = get_resnet_model(activation_function_type, n_classes)
    else:
        raise Exception("Currently we only support lenet or resnet50")
    
    return model
    

### Supported Post-Hoc OODN Processors

#### The first post processor we consider is ODIN

In [42]:
class OODPostprocessor():
    
    def inference(self, net: nn.Module, data_loader: DataLoader):
        pred_list, conf_list, label_list = [], [], []
        for idx, loaded_data in enumerate(data_loader):
            data, label = loaded_data[0], loaded_data[1]
            if idx % 50 == 0:
                print(f'Performing inference on batch: {idx}')
            pred, conf = self.postprocess(net, data.to(device))
            for idx in range(len(data)):
                pred_list.append(pred[idx].tolist())
                conf_list.append(conf[idx].tolist())
                label_list.append(label[idx].tolist())

        # convert values into numpy array
        pred_list = np.array(pred_list, dtype=int)
        conf_list = np.array(conf_list)
        label_list = np.array(label_list, dtype=int)

        return pred_list, conf_list, label_list

In [50]:
class ODINPostprocessor(OODPostprocessor):
    def __init__(self, temperature, noise):
        super(OODPostprocessor)
        self.temperature = temperature
        self.noise = noise
        
    def postprocess(self, net: nn.Module, data):
        net.eval()
        data.requires_grad = True
        output = net(data)

        # Calculating the perturbation we need to add, that is,
        # the sign of gradient of cross entropy loss w.r.t. input
        criterion = nn.CrossEntropyLoss()

        labels = output.detach().argmax(axis=1)

        # Using temperature scaling
        output = output / self.temperature

        loss = criterion(output, labels)
        loss.backward()

        # Normalizing the gradient to binary in {0, 1}
        gradient = torch.ge(data.grad.detach(), 0)
        gradient = (gradient.float() - 0.5) * 2

        # Scaling values taken from original code       
        gradient[:, 0] = (gradient[:, 0]) / (63.0 / 255.0)
        if gradient.shape[1] == 3:
            gradient[:, 1] = (gradient[:, 1]) / (62.1 / 255.0)
            gradient[:, 2] = (gradient[:, 2]) / (66.7 / 255.0)

        # Adding small perturbations to images
        tempInputs = torch.add(data.detach(), gradient, alpha=-self.noise)
        output = net(tempInputs)
        output = output / self.temperature

        # Calculating the confidence after adding perturbations
        nnOutput = output.detach()
        nnOutput = nnOutput - nnOutput.max(dim=1, keepdims=True).values
        nnOutput = nnOutput.exp() / nnOutput.exp().sum(dim=1, keepdims=True)

        conf, pred = nnOutput.max(dim=1)

        return pred, conf

#### We consider the Maximum Classifier Discrepancy Post OODN method

https://arxiv.org/pdf/1712.02560.pdf

In [51]:
class MCDPostprocessor(OODPostprocessor):
    def __init__(self, samples: int = 30):
        super(OODPostprocessor)
        self.samples = samples  #: number :math:`N` of samples

    def postprocess(self, model: torch.nn.Module, x: torch.Tensor) -> torch.Tensor:
        mode_switch = False
        if not model.training:
            mode_switch = True

            model.train()

            for mod in model.modules():
                # reset batch norm layers.
                # TODO: are there other layers?
                if isinstance(mod, (nn.BatchNorm1d, nn.BatchNorm2d, nn.BatchNorm3d)):
                    mod.train(False)

        results = None
        with torch.no_grad():
            for i in range(self.samples):
                output = model(x).softmax(dim=1)
                if results is None:
                    results = torch.zeros(size=output.shape).to(device)
                results += output
        results /= self.samples

        if mode_switch:
            model.eval()
        
        conf, pred = results.max(dim=1)

        return pred, conf

In [52]:
def get_postprocessor(postprocessor_type="odin"):
    if postprocessor_type == "odin":
        postprocessor = ODINPostprocessor(1000, 0.0014)
    elif postprocessor_type == "mcd":
        postprocessor = MCDPostprocessor(30)
    return postprocessor

### Supported Out of Distribution Detection Metrics

What metrics do we specifically care about here?

**FPR@95** measures the false positive rate (FPR) when the true positive rate (TPR) is
equal to 95%. Lower scores indicate better performance. 

**AUROC** measures the area under the
Receiver Operating Characteristic (ROC) curve, which displays the relationship between TPR and
FPR. The area under the ROC curve can be interpreted as the probability that a positive ID example
will have a higher detection score than a negative OOD example. 

**AUPR** measures the area under
the Precision-Recall (PR) curve. The PR curve is created by plotting precision versus recall. Similar
to AUROC, we consider ID samples as positive, so that the score corresponds to the AUPR-In metric
in some works

In [23]:
def calculate_oodn_metrics(model, postprocessor_type, id_test_loader, ood_test_loader, ood_name):
    postprocessor = get_postprocessor(postprocessor_type)
    id_pred, id_conf, id_gt = postprocessor.inference(
                model, id_test_loader)

    ood_pred, ood_conf, ood_gt = postprocessor.inference(
        model, ood_test_loader)

    ood_gt = -1 * np.ones_like(ood_gt)  # hard set to -1 as ood
    pred = np.concatenate([id_pred, ood_pred])
    conf = np.concatenate([id_conf, ood_conf])
    label = np.concatenate([id_gt, ood_gt])
    ood_metrics = metrics.compute_all_metrics(conf, label, pred)

    return print_and_get_formatted_metrics(ood_metrics, ood_name)

def print_and_get_formatted_metrics(metrics, dataset_name):
    [fpr, auroc, aupr_in, aupr_out,
     ccr_4, ccr_3, ccr_2, ccr_1, accuracy] \
     = metrics

    write_content = {
        'dataset': dataset_name,
        'FPR@95': '{:.2f}'.format(100 * fpr),
        'AUROC': '{:.2f}'.format(100 * auroc),
        'AUPR_IN': '{:.2f}'.format(100 * aupr_in),
        'AUPR_OUT': '{:.2f}'.format(100 * aupr_out),
        'CCR_4': '{:.2f}'.format(100 * ccr_4),
        'CCR_3': '{:.2f}'.format(100 * ccr_3),
        'CCR_2': '{:.2f}'.format(100 * ccr_2),
        'CCR_1': '{:.2f}'.format(100 * ccr_1),
        'ACC': '{:.2f}'.format(100 * accuracy)
    }

    fieldnames = list(write_content.keys())

    # print ood metric results
    print('FPR@95: {:.2f}, AUROC: {:.2f}'.format(100 * fpr, 100 * auroc),
          end=' ',
          flush=True)
    print('AUPR_IN: {:.2f}, AUPR_OUT: {:.2f}'.format(
        100 * aupr_in, 100 * aupr_out),
          flush=True)
    print('CCR: {:.2f}, {:.2f}, {:.2f}, {:.2f},'.format(
        ccr_4 * 100, ccr_3 * 100, ccr_2 * 100, ccr_1 * 100),
          end=' ',
          flush=True)
    print('ACC: {:.2f}'.format(accuracy * 100), flush=True)
    print(u'\u2500' * 70, flush=True)
    return write_content

def load_results_into_df(dir_path):
    res_files = [dir_path+each for each in listdir(dir_path)]
    all_results = []
    columns = ['optimizer_type', 'activation_function_type', 'postprocessor_type', 'trial', 'AUROC']
    for fp in res_files:
        f = open(fp)
        data = json.load(f)
        for trial, results in data.items():
            all_results.append([
                    results['optimizer_type'],
                    results['activation_function_type'],
                    results['postprocessor_type'],
                    trial,
                    float(results['AUROC'])
                ])
    df = pd.DataFrame(all_results, columns=columns)
    return df

In [24]:
def get_optimizer(model, config):
    params = model.parameters()
    lr = config['lr']
    momentum = config['momentum']
    weight_decay = config['weight_decay']
    optimizer_type = config['optimizer_type']
    
    print(f'Getting optimizer for type: {optimizer_type}...')
    if optimizer_type == 'SGD':
        return SGD(params, 
              lr=lr, 
              momentum=momentum,
              weight_decay=weight_decay)
    elif optimizer_type == 'Adam':
        return Adam(params, 
                    lr=lr, 
                    weight_decay=weight_decay)
    else:
        raise Exception("Invalid optimizer_type provided, only SGD and Adam are supported currently")

def get_wilds_loader(dataset, split, batch_size):
    d = dataset.get_subset(
        split,
        frac=0.1,
        transform=transforms.Compose(
            [transforms.Resize((448, 448)), transforms.ToTensor()]
        ),
    )
    # Prepare the standard data loader
    return get_train_loader("standard", d, batch_size=batch_size, num_workers=4, pin_memory=True)

def get_data_loaders(config):
    data_loaders = {}
    dataset_name = config["dataset_name"]
    dataset_type = config["dataset_type"]
    batch_size = config['batch_size']
    
    wilds_id_test_split = "id_val" if dataset_name == "camelyon17" else "id_test"
    if dataset_type == "wilds":
        # wilds dataset
        dataset = get_dataset(dataset=dataset_name, download=True)
        
        data_loaders["train"] = get_wilds_loader(dataset, "train", batch_size)
        data_loaders["ood_test"] = get_wilds_loader(dataset, "test", batch_size)
        data_loaders["id_test"] = get_wilds_loader(dataset, wilds_id_test_split, batch_size)
    elif dataset_name == "mnist":
        # mnist dataset
        train_dataset = mnist.MNIST(root='data', download=True, train=True, transform=ToTensor())
        test_dataset = mnist.MNIST(root='data', download=True, train=False, transform=ToTensor())
        fashion_test_dataset = mnist.FashionMNIST(root='data', download=True,train=False,transform=ToTensor())

        data_loaders["train"] = DataLoader(train_dataset, batch_size=batch_size)
        data_loaders["id_test"] = DataLoader(test_dataset, batch_size=batch_size)
        data_loaders["ood_test"] = DataLoader(fashion_test_dataset, batch_size=batch_size)
    
    return data_loaders

In [25]:
def train_resnet_model_given_opti_activation_fn(config):
    # get the train loader
    train_loader = config["data_loaders"]["train"]
    
    # get the resnet model with the replaced activation functions
    model = get_model(config)
    model.to(device)
    
    # get the optimizer
    sgd = get_optimizer(model, config)
    
    loss_fn = CrossEntropyLoss()

    for current_epoch in range(config['epochs']):
        model.train()
        epoch_start = time.time()
        print('Training epoch: {}'.format(current_epoch))
        
        time_per_hundred = time.time()
        for idx, (loader_data) in enumerate(train_loader):
            train_x, train_label = loader_data[0].cuda(), loader_data[1].cuda()
            sgd.zero_grad()
            predict_y = model(train_x.float())
            loss = loss_fn(predict_y, train_label.long())
            if idx % 100 == 0:
                print('idx: {}, loss: {}, time taken: {}'.format(idx, loss.sum().item(), time.time()-time_per_hundred))
                time_per_hundred = time.time()
            loss.backward()
            sgd.step()
        print(f"Time take for epoch {current_epoch}: {time.time() - epoch_start}s")
    
    torch.save(model, config['model_name'])
    return model

def run_full_oodn_pipeline(config):
    metrics = {}
    for i in range(config["trials"]):
        t = time.time()
        model_name = f"models/{config['dataset_name']}_{config['network']}_{config['postprocessor_type']}_{config['activation_function_type']}_{config['optimizer_type']}_{i}.pkl"
        print(f'Running model: {model_name}...')
        config['model_name'] = model_name
        # train model
        model = train_resnet_model_given_opti_activation_fn(config)
        # calculate oodn metrics
        metrics[i] = calculate_oodn_metrics(model,
                               config['postprocessor_type'],
                               config["data_loaders"]["id_test"], 
                               config["data_loaders"]["ood_test"], 
                               config["dataset_name"])
        metrics[i]['optimizer_type'] = config['optimizer_type']
        metrics[i]['activation_function_type'] = config['activation_function_type']
        metrics[i]['postprocessor_type'] = config['postprocessor_type']
        metrics[i]['time_taken'] = time.time() - t
        print(f"Time taken to train: {metrics[i]['time_taken']}")
        
    experiment_name = f"{config['results_dir']}/{config['dataset_name']}_{config['network']}_{config['postprocessor_type']}_{config['activation_function_type']}_{config['optimizer_type']}.json"
    with open(experiment_name, 'w') as fp:
        json.dump(metrics, fp)
    return metrics

### Study 1: LeNet5, MNIST as the ID Dataset, FashionMNIST as the OOD Dataset

#### Study 1(a): Combination: SGD + ReLU

#### ODIN

In [55]:
config_1a = {
    "batch_size": 128,
    "n_classes": 10,
    "dataset_name": "mnist",
    "epochs": 10,
    "version": time.time(),
    "lr": 0.1,
    "momentum": 0.9,
    "weight_decay": 0.0005,
    "optimizer_type": "SGD",
    "activation_function_type": "relu",
    "network": "lenet",
    "postprocessor_type": "odin",
    "dataset_type": "mnist",
    "trials": 3,
    "results_dir": "mnist-study"
}
config_1a["data_loaders"] = get_data_loaders(config_1a)
run_full_oodn_pipeline(config_1a)

#### MCD

In [26]:
config_1_1a = {
    "batch_size": 128,
    "n_classes": 10,
    "dataset_name": "mnist",
    "epochs": 10,
    "version": time.time(),
    "lr": 0.1,
    "momentum": 0.9,
    "weight_decay": 0.0005,
    "optimizer_type": "SGD",
    "activation_function_type": "relu",
    "network": "lenet",
    "postprocessor_type": "mcd",
    "dataset_type": "mnist",
    "trials": 3,
    "results_dir": "mnist-study"
}
config_1_1a["data_loaders"] = get_data_loaders(config_1_1a)
run_full_oodn_pipeline(config_1_1a)

Running model: models/mnist_lenet_mcd_relu_SGD_0.pkl...
Getting optimizer for type: SGD...
Training epoch: 0
idx: 0, loss: 2.2967536449432373, time taken: 0.00898599624633789
idx: 100, loss: 0.2458113431930542, time taken: 0.9537761211395264
idx: 200, loss: 0.3505885601043701, time taken: 0.9808645248413086
idx: 300, loss: 0.09110728651285172, time taken: 0.9493944644927979
idx: 400, loss: 0.23323051631450653, time taken: 0.9486937522888184
Time take for epoch 0: 4.483798027038574s
Training epoch: 1
idx: 0, loss: 0.07201088964939117, time taken: 0.008733272552490234
idx: 100, loss: 0.11222241073846817, time taken: 0.9483945369720459
idx: 200, loss: 0.06680341809988022, time taken: 0.9406368732452393
idx: 300, loss: 0.06127895787358284, time taken: 0.9487392902374268
idx: 400, loss: 0.3410691022872925, time taken: 0.9534344673156738
Time take for epoch 1: 4.442000865936279s
Training epoch: 2
idx: 0, loss: 0.055400170385837555, time taken: 0.008737564086914062
idx: 100, loss: 0.022906618

TypeError: postprocess() takes 2 positional arguments but 3 were given

#### 1 (b) SGD + SoftPlus

In [13]:
config_1b = {
    "batch_size": 128,
    "dataset_name": "mnist",
    "n_classes": 10,
    "epochs": 10,
    "version": time.time(),
    "lr": 0.1,
    "momentum": 0.9,
    "weight_decay": 0.0005,
    "optimizer_type": "SGD",
    "activation_function_type": "softplus",
    "network": "lenet",
    "postprocessor_type": "odin",
    "dataset_type": "mnist",
    "trials": 3,
    "results_dir": "mnist-study"
}
config_1b["data_loaders"] = get_data_loaders(config_1b)
run_full_oodn_pipeline(config_1b)

/opt/conda/lib/python3.7/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Running model: models/lenet_mnist_softplus_SGD_0.pkl...
Getting optimizer for type: SGD...
Training epoch: 0
idx: 0, loss: 2.3213627338409424
idx: 100, loss: 2.2994141578674316
idx: 200, loss: 0.6801586151123047
idx: 300, loss: 0.43440037965774536
idx: 400, loss: 0.39628586173057556
Training epoch: 1
idx: 0, loss: 0.11309085041284561
idx: 100, loss: 0.14323793351650238
idx: 200, loss: 0.15335214138031006
idx: 300, loss: 0.09007233381271362
idx: 400, loss: 0.4375433027744293
Training epoch: 2
idx: 0, loss: 0.06423605978488922
idx: 100, loss: 0.16616109013557434
idx: 200, loss: 0.12796740233898163
idx: 300, loss: 0.0667390376329422
idx: 400, loss: 0.22158698737621307
Training epoch: 3
idx: 0, loss: 0.08575321733951569
idx: 100, loss: 0.10658863186836243
idx: 200, loss: 0.10456061363220215
idx: 300, loss: 0.033075228333473206
idx: 400, loss: 0.17125645279884338
Training epoch: 4
idx: 0, loss: 0.06995586305856705
idx: 100, loss: 0.08943136781454086
idx: 200, loss: 0.1137051209807396
idx: 3

{0: {'dataset': 'mnist',
  'FPR@95': '9.36',
  'AUROC': '98.15',
  'AUPR_IN': '98.37',
  'AUPR_OUT': '97.87',
  'CCR_4': '21.26',
  'CCR_3': '56.62',
  'CCR_2': '79.24',
  'CCR_1': '94.45',
  'ACC': '98.18',
  'optimizer_type': 'SGD',
  'activation_function_type': 'softplus',
  'postprocessor_type': 'odin'},
 1: {'dataset': 'mnist',
  'FPR@95': '1.06',
  'AUROC': '99.63',
  'AUPR_IN': '99.67',
  'AUPR_OUT': '99.60',
  'CCR_4': '46.94',
  'CCR_3': '86.31',
  'CCR_2': '94.22',
  'CCR_1': '98.12',
  'ACC': '98.57',
  'optimizer_type': 'SGD',
  'activation_function_type': 'softplus',
  'postprocessor_type': 'odin'},
 2: {'dataset': 'mnist',
  'FPR@95': '3.42',
  'AUROC': '99.15',
  'AUPR_IN': '99.23',
  'AUPR_OUT': '99.07',
  'CCR_4': '18.20',
  'CCR_3': '69.25',
  'CCR_2': '88.15',
  'CCR_1': '97.16',
  'ACC': '98.36',
  'optimizer_type': 'SGD',
  'activation_function_type': 'softplus',
  'postprocessor_type': 'odin'}}

In [18]:
config_1_1b = {
    "batch_size": 128,
    "dataset_name": "mnist",
    "n_classes": 10,
    "epochs": 10,
    "version": time.time(),
    "lr": 0.1,
    "momentum": 0.9,
    "weight_decay": 0.0005,
    "optimizer_type": "SGD",
    "activation_function_type": "softplus",
    "network": "lenet",
    "postprocessor_type": "mcd",
    "dataset_type": "mnist",
    "trials": 3,
    "results_dir": "mnist-study"
}
config_1_1b["data_loaders"] = get_data_loaders(config_1_1b)
run_full_oodn_pipeline(config_1_1b)

Running model: models/mnist_lenet_mcd_softplus_SGD_0.pkl...
Getting optimizer for type: SGD...
Training epoch: 0
idx: 0, loss: 2.3624277114868164
idx: 100, loss: 2.3057777881622314
idx: 200, loss: 2.442692995071411
idx: 300, loss: 0.5746510624885559
idx: 400, loss: 0.4224349558353424
Training epoch: 1
idx: 0, loss: 0.17665119469165802
idx: 100, loss: 0.2759348452091217
idx: 200, loss: 0.3475848436355591
idx: 300, loss: 0.12519679963588715
idx: 400, loss: 0.19080723822116852
Training epoch: 2
idx: 0, loss: 0.14175932109355927
idx: 100, loss: 0.15821029245853424
idx: 200, loss: 0.16171494126319885
idx: 300, loss: 0.06106771528720856
idx: 400, loss: 0.24474868178367615
Training epoch: 3
idx: 0, loss: 0.11062025278806686
idx: 100, loss: 0.17143307626247406
idx: 200, loss: 0.09735853224992752
idx: 300, loss: 0.047896698117256165
idx: 400, loss: 0.25606343150138855
Training epoch: 4
idx: 0, loss: 0.05433105304837227
idx: 100, loss: 0.05655233934521675
idx: 200, loss: 0.10700041055679321
idx:

{0: {'dataset': 'mnist',
  'FPR@95': '3.74',
  'AUROC': '99.19',
  'AUPR_IN': '99.28',
  'AUPR_OUT': '99.12',
  'CCR_4': '56.84',
  'CCR_3': '74.26',
  'CCR_2': '89.46',
  'CCR_1': '96.09',
  'ACC': '97.55',
  'optimizer_type': 'SGD',
  'activation_function_type': 'softplus',
  'postprocessor_type': 'mcd'},
 1: {'dataset': 'mnist',
  'FPR@95': '7.70',
  'AUROC': '98.77',
  'AUPR_IN': '98.91',
  'AUPR_OUT': '98.66',
  'CCR_4': '56.58',
  'CCR_3': '76.99',
  'CCR_2': '85.89',
  'CCR_1': '95.25',
  'ACC': '97.84',
  'optimizer_type': 'SGD',
  'activation_function_type': 'softplus',
  'postprocessor_type': 'mcd'},
 2: {'dataset': 'mnist',
  'FPR@95': '17.49',
  'AUROC': '96.35',
  'AUPR_IN': '96.41',
  'AUPR_OUT': '96.39',
  'CCR_4': '13.01',
  'CCR_3': '37.71',
  'CCR_2': '57.03',
  'CCR_1': '88.70',
  'ACC': '97.96',
  'optimizer_type': 'SGD',
  'activation_function_type': 'softplus',
  'postprocessor_type': 'mcd'}}

#### 1(c) Adam + ReLU

In [56]:
config_1c = {
    "batch_size": 128,
    "dataset_name": "mnist",
    "n_classes": 10,
    "epochs": 10,
    "version": time.time(),
    "lr": 0.01,
    "momentum": 0.9,
    "weight_decay": 0.0005,
    "optimizer_type": "Adam",
    "activation_function_type": "relu",
    "network": "lenet",
    "postprocessor_type": "odin",
    "dataset_type": "mnist",
    "trials": 3,
    "results_dir": "mnist-study"
}
config_1c["data_loaders"] = get_data_loaders(config_1c)
run_full_oodn_pipeline(config_1c)

Running model: models/lenet_mnist_relu_Adam_0.pkl...
Getting optimizer for type: Adam...
Training epoch: 0
idx: 0, loss: 2.301894187927246
idx: 100, loss: 0.11984912306070328
idx: 200, loss: 0.15497374534606934
idx: 300, loss: 0.06791342049837112
idx: 400, loss: 0.22912515699863434
Training epoch: 1
idx: 0, loss: 0.07157646864652634
idx: 100, loss: 0.05058322101831436
idx: 200, loss: 0.09429037570953369
idx: 300, loss: 0.08827465027570724
idx: 400, loss: 0.1425236165523529
Training epoch: 2
idx: 0, loss: 0.08118437230587006
idx: 100, loss: 0.029627040028572083
idx: 200, loss: 0.11240573972463608
idx: 300, loss: 0.049749165773391724
idx: 400, loss: 0.10378781706094742
Training epoch: 3
idx: 0, loss: 0.05978626012802124
idx: 100, loss: 0.04203808680176735
idx: 200, loss: 0.12809190154075623
idx: 300, loss: 0.072877898812294
idx: 400, loss: 0.12331315875053406
Training epoch: 4
idx: 0, loss: 0.04495770111680031
idx: 100, loss: 0.02584172785282135
idx: 200, loss: 0.11872470378875732
idx: 3

{0: {'dataset': 'mnist',
  'FPR@95': '2.70',
  'AUROC': '99.40',
  'AUPR_IN': '99.44',
  'AUPR_OUT': '99.38',
  'CCR_4': '52.68',
  'CCR_3': '74.63',
  'CCR_2': '91.00',
  'CCR_1': '97.29',
  'ACC': '98.42',
  'optimizer_type': 'Adam',
  'activation_function_type': 'relu',
  'postprocessor_type': 'odin'},
 1: {'dataset': 'mnist',
  'FPR@95': '2.21',
  'AUROC': '99.41',
  'AUPR_IN': '99.47',
  'AUPR_OUT': '99.36',
  'CCR_4': '73.34',
  'CCR_3': '81.76',
  'CCR_2': '91.29',
  'CCR_1': '97.00',
  'ACC': '97.89',
  'optimizer_type': 'Adam',
  'activation_function_type': 'relu',
  'postprocessor_type': 'odin'},
 2: {'dataset': 'mnist',
  'FPR@95': '0.97',
  'AUROC': '99.55',
  'AUPR_IN': '99.61',
  'AUPR_OUT': '99.51',
  'CCR_4': '73.59',
  'CCR_3': '89.21',
  'CCR_2': '94.54',
  'CCR_1': '97.55',
  'ACC': '98.29',
  'optimizer_type': 'Adam',
  'activation_function_type': 'relu',
  'postprocessor_type': 'odin'}}

In [19]:
config_1_1c = {
    "batch_size": 128,
    "dataset_name": "mnist",
    "n_classes": 10,
    "epochs": 10,
    "version": time.time(),
    "lr": 0.01,
    "momentum": 0.9,
    "weight_decay": 0.0005,
    "optimizer_type": "Adam",
    "activation_function_type": "relu",
    "network": "lenet",
    "postprocessor_type": "mcd",
    "dataset_type": "mnist",
    "trials": 3,
    "results_dir": "mnist-study"
}
config_1_1c["data_loaders"] = get_data_loaders(config_1_1c)
run_full_oodn_pipeline(config_1_1c)

Running model: models/mnist_lenet_mcd_relu_Adam_0.pkl...
Getting optimizer for type: Adam...
Training epoch: 0
idx: 0, loss: 2.3105053901672363
idx: 100, loss: 0.16683584451675415
idx: 200, loss: 0.16353853046894073
idx: 300, loss: 0.11336103826761246
idx: 400, loss: 0.36516520380973816
Training epoch: 1
idx: 0, loss: 0.09431233257055283
idx: 100, loss: 0.047792594879865646
idx: 200, loss: 0.14638617634773254
idx: 300, loss: 0.09262651205062866
idx: 400, loss: 0.24459731578826904
Training epoch: 2
idx: 0, loss: 0.01773892156779766
idx: 100, loss: 0.04295041039586067
idx: 200, loss: 0.0693758875131607
idx: 300, loss: 0.05112284794449806
idx: 400, loss: 0.31428009271621704
Training epoch: 3
idx: 0, loss: 0.07498281449079514
idx: 100, loss: 0.04888544976711273
idx: 200, loss: 0.08386431634426117
idx: 300, loss: 0.07530947774648666
idx: 400, loss: 0.2098739743232727
Training epoch: 4
idx: 0, loss: 0.043862421065568924
idx: 100, loss: 0.02606273628771305
idx: 200, loss: 0.10874634981155396


{0: {'dataset': 'mnist',
  'FPR@95': '0.25',
  'AUROC': '99.70',
  'AUPR_IN': '99.75',
  'AUPR_OUT': '99.65',
  'CCR_4': '82.60',
  'CCR_3': '92.38',
  'CCR_2': '96.07',
  'CCR_1': '97.89',
  'ACC': '98.31',
  'optimizer_type': 'Adam',
  'activation_function_type': 'relu',
  'postprocessor_type': 'mcd'},
 1: {'dataset': 'mnist',
  'FPR@95': '2.73',
  'AUROC': '99.35',
  'AUPR_IN': '99.42',
  'AUPR_OUT': '99.29',
  'CCR_4': '71.59',
  'CCR_3': '78.44',
  'CCR_2': '90.85',
  'CCR_1': '97.29',
  'ACC': '98.28',
  'optimizer_type': 'Adam',
  'activation_function_type': 'relu',
  'postprocessor_type': 'mcd'},
 2: {'dataset': 'mnist',
  'FPR@95': '0.33',
  'AUROC': '99.75',
  'AUPR_IN': '99.78',
  'AUPR_OUT': '99.73',
  'CCR_4': '87.00',
  'CCR_3': '91.07',
  'CCR_2': '95.84',
  'CCR_1': '97.88',
  'ACC': '98.22',
  'optimizer_type': 'Adam',
  'activation_function_type': 'relu',
  'postprocessor_type': 'mcd'}}

#### 1(d) Adam + Softplus

In [57]:
config_1d = {
    "batch_size": 128,
    "dataset_name": "mnist",
    "n_classes": 10,
    "epochs": 10,
    "version": time.time(),
    "lr": 0.01,
    "momentum": 0.9,
    "weight_decay": 0.0005,
    "optimizer_type": "Adam",
    "activation_function_type": "softplus",
    "network": "lenet",
    "postprocessor_type": "odin",
    "dataset_type": "mnist",
    "trials": 3,
    "results_dir": "mnist-study"
}
config_1d["data_loaders"] = get_data_loaders(config_1d)
run_full_oodn_pipeline(config_1d)

Running model: models/lenet_mnist_softplus_Adam_0.pkl...
Getting optimizer for type: Adam...
Training epoch: 0
idx: 0, loss: 2.3255388736724854
idx: 100, loss: 2.3116369247436523
idx: 200, loss: 2.305769443511963
idx: 300, loss: 2.2983803749084473
idx: 400, loss: 2.307865619659424
Training epoch: 1
idx: 0, loss: 2.276770830154419
idx: 100, loss: 0.8078237175941467
idx: 200, loss: 0.25390756130218506
idx: 300, loss: 0.32934701442718506
idx: 400, loss: 0.3490292727947235
Training epoch: 2
idx: 0, loss: 0.1403365433216095
idx: 100, loss: 0.2735982835292816
idx: 200, loss: 0.14142420887947083
idx: 300, loss: 0.18107478320598602
idx: 400, loss: 0.2841825485229492
Training epoch: 3
idx: 0, loss: 0.10381539165973663
idx: 100, loss: 0.18071672320365906
idx: 200, loss: 0.11766538769006729
idx: 300, loss: 0.14970530569553375
idx: 400, loss: 0.25166428089141846
Training epoch: 4
idx: 0, loss: 0.13393616676330566
idx: 100, loss: 0.14538753032684326
idx: 200, loss: 0.12768563628196716
idx: 300, los

{0: {'dataset': 'mnist',
  'FPR@95': '30.70',
  'AUROC': '93.93',
  'AUPR_IN': '94.30',
  'AUPR_OUT': '93.41',
  'CCR_4': '3.42',
  'CCR_3': '20.72',
  'CCR_2': '50.00',
  'CCR_1': '81.76',
  'ACC': '95.28',
  'optimizer_type': 'Adam',
  'activation_function_type': 'softplus',
  'postprocessor_type': 'odin'},
 1: {'dataset': 'mnist',
  'FPR@95': '5.51',
  'AUROC': '98.81',
  'AUPR_IN': '98.96',
  'AUPR_OUT': '98.67',
  'CCR_4': '43.70',
  'CCR_3': '68.11',
  'CCR_2': '86.62',
  'CCR_1': '95.13',
  'ACC': '97.47',
  'optimizer_type': 'Adam',
  'activation_function_type': 'softplus',
  'postprocessor_type': 'odin'},
 2: {'dataset': 'mnist',
  'FPR@95': '39.54',
  'AUROC': '93.00',
  'AUPR_IN': '93.58',
  'AUPR_OUT': '92.81',
  'CCR_4': '2.51',
  'CCR_3': '19.59',
  'CCR_2': '49.86',
  'CCR_1': '78.18',
  'ACC': '94.28',
  'optimizer_type': 'Adam',
  'activation_function_type': 'softplus',
  'postprocessor_type': 'odin'}}

In [20]:
config_1_1d = {
    "batch_size": 128,
    "dataset_name": "mnist",
    "n_classes": 10,
    "epochs": 10,
    "version": time.time(),
    "lr": 0.01,
    "momentum": 0.9,
    "weight_decay": 0.0005,
    "optimizer_type": "Adam",
    "activation_function_type": "softplus",
    "network": "lenet",
    "postprocessor_type": "mcd",
    "dataset_type": "mnist",
    "trials": 3,
    "results_dir": "mnist-study"
}
config_1_1d["data_loaders"] = get_data_loaders(config_1_1d)
run_full_oodn_pipeline(config_1_1d)

Running model: models/mnist_lenet_mcd_softplus_Adam_0.pkl...
Getting optimizer for type: Adam...
Training epoch: 0
idx: 0, loss: 2.477346897125244
idx: 100, loss: 2.309143543243408
idx: 200, loss: 2.3063406944274902
idx: 300, loss: 2.297903060913086
idx: 400, loss: 1.1604360342025757
Training epoch: 1
idx: 0, loss: 0.37056636810302734
idx: 100, loss: 0.36415427923202515
idx: 200, loss: 0.14169979095458984
idx: 300, loss: 0.20955771207809448
idx: 400, loss: 0.40641340613365173
Training epoch: 2
idx: 0, loss: 0.19860942661762238
idx: 100, loss: 0.16836953163146973
idx: 200, loss: 0.11009348183870316
idx: 300, loss: 0.08757030218839645
idx: 400, loss: 0.3184612989425659
Training epoch: 3
idx: 0, loss: 0.07341178506612778
idx: 100, loss: 0.10277615487575531
idx: 200, loss: 0.09509484469890594
idx: 300, loss: 0.09184584766626358
idx: 400, loss: 0.25116777420043945
Training epoch: 4
idx: 0, loss: 0.1025996208190918
idx: 100, loss: 0.07905026525259018
idx: 200, loss: 0.06830193847417831
idx: 

{0: {'dataset': 'mnist',
  'FPR@95': '2.74',
  'AUROC': '99.29',
  'AUPR_IN': '99.37',
  'AUPR_OUT': '99.20',
  'CCR_4': '57.37',
  'CCR_3': '72.99',
  'CCR_2': '89.96',
  'CCR_1': '95.97',
  'ACC': '97.12',
  'optimizer_type': 'Adam',
  'activation_function_type': 'softplus',
  'postprocessor_type': 'mcd'},
 1: {'dataset': 'mnist',
  'FPR@95': '19.76',
  'AUROC': '96.68',
  'AUPR_IN': '97.26',
  'AUPR_OUT': '95.92',
  'CCR_4': '31.55',
  'CCR_3': '52.66',
  'CCR_2': '73.68',
  'CCR_1': '90.83',
  'ACC': '98.20',
  'optimizer_type': 'Adam',
  'activation_function_type': 'softplus',
  'postprocessor_type': 'mcd'},
 2: {'dataset': 'mnist',
  'FPR@95': '5.64',
  'AUROC': '98.84',
  'AUPR_IN': '98.99',
  'AUPR_OUT': '98.61',
  'CCR_4': '35.22',
  'CCR_3': '64.05',
  'CCR_2': '86.40',
  'CCR_1': '95.95',
  'ACC': '97.98',
  'optimizer_type': 'Adam',
  'activation_function_type': 'softplus',
  'postprocessor_type': 'mcd'}}

### Robustness Analysis For Study 1

In [21]:
df_study_1 = load_results_into_df('mnist-study/')
df_study_1

,optimizer_type,activation_function_type,postprocessor_type,trial,AUROC
0,SGD,softplus,odin,0,98.15
1,SGD,softplus,odin,1,99.63
2,SGD,softplus,odin,2,99.15
3,Adam,softplus,odin,0,93.93
4,Adam,softplus,odin,1,98.81
5,Adam,softplus,odin,2,93.00
6,Adam,relu,mcd,0,99.70
7,Adam,relu,mcd,1,99.35
8,Adam,relu,mcd,2,99.75
9,SGD,relu,mcd,0,99.15


In [22]:
df_study_1.groupby(['optimizer_type', 'activation_function_type'])['AUROC'].describe()

count       mean       std    min  \
optimizer_type activation_function_type                                      
Adam           relu                        6.0  99.526667  0.168127  99.35   
               softplus                    6.0  96.758333  2.723332  93.00   
SGD            relu                        6.0  99.295000  0.387234  98.71   
               softplus                    6.0  98.540000  1.181846  96.35   

                                             25%     50%      75%    max  
optimizer_type activation_function_type                                   
Adam           relu                      99.4025  99.480  99.6625  99.75  
               softplus                  94.6175  97.745  98.8325  99.29  
SGD            relu                      99.1425  99.255  99.5400  99.81  
               softplus                  98.3050  98.960  99.1800  99.63

In [23]:
df_study_1.groupby(['activation_function_type'])['AUROC'].describe()

,count,mean,std,min,25%,50%,75%,max
activation_function_type,,,,,,,,
relu,12.0,99.410833,0.309265,98.71,99.3000,99.405,99.625,99.81
softplus,12.0,97.649167,2.207209,93.00,96.5975,98.790,99.160,99.63


In [24]:
df_study_1.groupby(['optimizer_type'])['AUROC'].describe()

,count,mean,std,min,25%,50%,75%,max
optimizer_type,,,,,,,,
Adam,12.0,98.1425,2.339678,93.00,98.2775,99.32,99.445,99.75
SGD,12.0,98.9175,0.926559,96.35,98.7550,99.15,99.420,99.81


In [53]:
def perform_oodn_analysis_on_preloaded_model(model_path, config):
    model = torch.load(model_path)
    calculate_oodn_metrics(model,
                                   config['postprocessor_type'],
                                   config["data_loaders"]["id_test"],
                                   config["data_loaders"]["ood_test"],
                                   config["dataset_name"])

In [54]:
perform_oodn_analysis_on_preloaded_model('models/lenet_mnist_softplus_Adam_0.pkl', config_1_1a)

Performing inference on batch: 0
Performing inference on batch: 50
Performing inference on batch: 0
Performing inference on batch: 50
FPR@95: 45.00, AUROC: 92.68 AUPR_IN: 93.77, AUPR_OUT: 91.00
CCR: 1.08, 25.41, 51.53, 82.04, ACC: 95.29
──────────────────────────────────────────────────────────────────────


In [56]:
perform_oodn_analysis_on_preloaded_model('models/lenet_mnist_softplus_Adam_0.pkl', config_1a)

Performing inference on batch: 0
Performing inference on batch: 50
Performing inference on batch: 0
Performing inference on batch: 50
FPR@95: 30.70, AUROC: 93.93 AUPR_IN: 94.30, AUPR_OUT: 93.41
CCR: 3.44, 20.72, 50.02, 81.76, ACC: 95.28
──────────────────────────────────────────────────────────────────────
